<a href="https://colab.research.google.com/github/kingsman0909/MyPortfolio/blob/main/bark_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 🚀 Install Bark and dependencies
!pip install -q git+https://github.com/camenduru/bark.git
!pip install -q --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118
!pip install -q gradio scipy

# 📦 Download Bark model weights (only required ones)
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/bark_v0/resolve/main/6285677e88715abde42a9924db939b3b.pt -d /root/.cache/suno/bark_v0 -o 6285677e88715abde42a9924db939b3b.pt

# 🛠 Patch torch.load to allow model loading
import torch
_original_load = torch.load
def custom_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return _original_load(*args, **kwargs)
torch.load = custom_load

# 🔊 Generate WAV from input text and speaker
from bark import SAMPLE_RATE, generate_audio, preload_models
from bark.generation import SUPPORTED_LANGS
from scipy.io.wavfile import write as write_wav
import numpy as np

_ = preload_models()

# 🎤 Available voices
PROMPT_LOOKUP = {"Unconditional": None, "Announcer": "announcer"}
AVAILABLE_PROMPTS = ["Unconditional", "Announcer"]

for _, lang in SUPPORTED_LANGS:
    for i in range(10):
        label = f"Speaker {i} ({lang})"
        PROMPT_LOOKUP[label] = f"{lang}_speaker_{i}"
        AVAILABLE_PROMPTS.append(label)

print("✅ Available voice prompts:")
for voice in AVAILABLE_PROMPTS:
    print(" -", voice)

# 📝 Custom settings: MODIFY THESE
TEXT = "This is a test voice line using Bark."
VOICE = "Speaker 1 (en)"   # 👈 Change this (e.g., "Announcer", "Speaker 3 (de)", "Unconditional")
OUTPUT_PATH = "bark_output.wav"

# 🎧 Generate & save WAV
voice_id = PROMPT_LOOKUP[VOICE]
audio_array = generate_audio(TEXT, history_prompt=voice_id)
audio_array = (audio_array * 32767).astype(np.int16)
write_wav(OUTPUT_PATH, SAMPLE_RATE, audio_array)

print(f"✅ Audio saved to {OUTPUT_PATH}")


ERROR: Operation cancelled by user
  Installing build dependencies ... Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 1

KeyboardInterrupt: 